# Asynchronous Actor-Critic Agents (A3C)

An implementation of the A3C algorithm to solve a simple 3D Doom challenge using the VizDoom engine.

The A3C algorithm was released by Google’s DeepMind group earlier this year, and it made a splash by essentially obsoleting DQN. It was faster, simpler, more robust, and able to achieve much better scores on the standard battery of Deep RL tasks. On top of all that it could work in continuous as well as discrete action spaces. Given this, it has become the go-to Deep RL algorithm for new challenging problems with complex state and action spaces. In fact, OpenAI just released a version of A3C as their “universal starter agent” for working with their new (and very diverse) set of Universe environments.

Asynchronous. Unlike DQN, where a single agent represented by a single neural network interacts with a single environment, A3C utilizes multiple incarnations of the above in order to learn more efficiently. In A3C there is a global network, and multiple worker agents which each have their own set of network parameters. Each of these agents interacts with it’s own copy of the environment at the same time as the other agents are interacting with their environments. The reason this works better than having a single agent (beyond the speedup of getting more work done), is that the experience of each agent is independent of the experience of the others. In this way the overall experience available for training becomes more diverse.

Actor-Critic. Actor-Critic combines the benefits of both value-iteration methods such as Q-learning, and policy-iteration methods such as Policy Gradient. In the case of A3C, our network will estimate both a value function V(s) (how good a certain state is to be in) and a policy π(s) (a set of action probability outputs). These will each be separate fully-connected layers sitting at the top of the network. Critically, the agent uses the value estimate (the critic) to update the policy (the actor) more intelligently than traditional policy gradient methods.

Advantage. The insight of using advantage estimates rather than just discounted returns is to allow the agent to determine not just how good its actions were, but how much better they turned out to be than expected. Intuitively, this allows the algorithm to focus on where the network’s predictions were lacking.

Advantage: A = Q(s,a) - V(s)

Since we won’t be determining the Q values directly in A3C, we can use the discounted returns (R) as an estimate of Q(s,a) to allow us to generate an estimate of the advantage.

Estimated Advantage: A = R - V(s)

This model utilizes a slightly different version of advantage estimation with lower variance referred to as Generalized Advantage Estimation.

In [1]:
#!pip install vizdoom

In [2]:
from helper import *
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
from random import choice
import scipy.signal
import tensorflow as tf
import tensorflow.contrib.slim as slim
import threading
from time import sleep, time
from vizdoom import *
%matplotlib inline

In [22]:
def update_target_graph(from_scope, to_scope):
    """
    Copies one set of variables to another. Used to set worker network parameters 
    to those of global network.
    """
    from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, from_scope)
    to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, to_scope)
    op_holder = []
    for from_var, to_var in zip(from_vars, to_vars):
        op_holder.append(to_var.assign(from_var))
        
    return op_holder

In [23]:
def process_frame(frame):
    """ Preprocess Doom screen image to produce cropped and resized image """
    s = frame[10:-10, 30:-30]
    s = scipy.misc.imresize(s, [84, 84])
    s = np.reshape(s, [np.prod(s.shape)]) / 255.
    return s

In [24]:
def discount(x, gamma):
    """ Calculate discounted returns """
    return scipy.signal.lfilter([1], [1, -gamma], x[::-1], axis=0)[::-1]

In [25]:
def normalized_columns_initializer(std=1.):
    """ Initialize weights for policy and value output layers """
    def _initializer(shape, dtype=None, partition_info=None):
        out = np.random.randn(*shape).astype(np.float32)
        out *= std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
        return tf.constant(out)
    
    return _initializer

In [26]:
class ACNetwork(object):
    
    def __init__(self, s_size, a_size, scope, optimizer):
        with tf.variable_scope(scope):
            # input and visual encoding layers
            self.inputs = tf.placeholder(shape=[None, s_size], dtype=tf.float32)
            self.image_in = tf.reshape(self.inputs, shape=[-1, 84, 84, 1])
            self.conv1 = slim.conv2d(activation_fn=tf.nn.elu, inputs=self.image_in, num_outputs=16, 
                                     kernel_size=[8, 8], stride=[4, 4], padding='VALID')
            self.conv2 = slim.conv2d(activation_fn=tf.nn.elu, inputs=self.conv1, num_outputs=32, 
                                     kernel_size=[4, 4], stride=[2, 2], padding='VALID')
            hidden = slim.fully_connected(slim.flatten(self.conv2), 256, activation_fn=tf.nn.elu)
            
            # Recurrent network for temporal dependencies
            lstm_cell = tf.contrib.rnn.BasicLSTMCell(256, state_is_tuple=True)
            c_init = np.zeros((1, lstm_cell.state_size.c), np.float32)
            h_init = np.zeros((1, lstm_cell.state_size.h), np.float32)
            self.state_init = [c_init, h_init]
            c_in = tf.placeholder(tf.float32, [1, lstm_cell.state_size.c])
            h_in = tf.placeholder(tf.float32, [1, lstm_cell.state_size.h])
            self.state_in = (c_in, h_in)
            rnn_in = tf.expand_dims(hidden, [0])
            step_size = tf.shape(self.image_in)[:1]
            state_in = tf.contrib.rnn.LSTMStateTuple(c_in, h_in)
            lstm_outputs, lstm_state = tf.nn.dynamic_rnn(lstm_cell, rnn_in, 
                                                         initial_state=state_in, 
                                                         sequence_length=step_size, 
                                                         time_major=False)
            lstm_c, lstm_h = lstm_state
            self.state_out = (lstm_c[:1, :], lstm_h[:1, :])
            rnn_out = tf.reshape(lstm_outputs, [-1, 256])
            
            # output layers for policy and value estimations
            self.policy = slim.fully_connected(rnn_out, a_size, 
                                               activation_fn=tf.nn.softmax, 
                                               weights_initializer=normalized_columns_initializer(0.01), 
                                               biases_initializer=None)
            self.value = slim.fully_connected(rnn_out, 1, 
                                              activation_fn=None, 
                                              weights_initializer=normalized_columns_initializer(1.0), 
                                              biases_initializer=None)
            
            # only the worker network needs ops for loss and gradient update
            if scope != 'global':
                self.actions = tf.placeholder(shape=[None], dtype=tf.int32)
                self.actions_onehot = tf.one_hot(self.actions, a_size, dtype=tf.float32)
                self.target_v = tf.placeholder(shape=[None], dtype=tf.float32)
                self.advantages = tf.placeholder(shape=[None], dtype=tf.float32)
                self.responsible_outputs = tf.reduce_sum(self.policy * self.actions_onehot, [1])
                
                # loss functions
                self.value_loss = 0.5 * tf.reduce_sum(tf.square(self.target_v - tf.reshape(self.value, [-1])))
                self.entropy = -tf.reduce_sum(self.policy * tf.log(self.policy))
                self.policy_loss = -tf.reduce_sum(tf.log(self.responsible_outputs) * self.advantages)
                self.loss = 0.5 * self.value_loss + self.policy_loss - self.entropy * 0.01
                
                # get gradients from local network using local losses
                local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
                self.gradients = tf.gradients(self.loss, local_vars)
                self.var_norms = tf.global_norm(local_vars)
                grads, self.grad_norms = tf.clip_by_global_norm(self.gradients, 40.)
                
                # apply local gradients to global network
                global_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'global')
                self.apply_grads = optimizer.apply_gradients(zip(grads, global_vars))

### Worker Agent

In [36]:
class Worker(object):
    
    def __init__(self, game, name, s_size, a_size, optimizer, model_path, global_episodes):
        self.name = 'worker_{}'.format(name)
        self.number = name
        self.model_path = model_path
        self.optimizer = optimizer
        self.global_episodes = global_episodes
        self.increment = self.global_episodes.assign_add(1)
        self.episode_rewards = []
        self.episode_lengths = []
        self.episode_mean_values = []
        self.summary_writer = tf.summary.FileWriter('train_{}'.format(self.number))
        
        # create the local copy of the network and the tensorflow op
        # to copy global parameters to the local network
        self.local_ac = ACNetwork(s_size, a_size, self.name, optimizer)
        self.update_local_ops = update_target_graph('global', self.name)
        
        # the following code is for setting up the Doom environment
        # this corresponds to the simple task we will pose our agent
        game.set_doom_scenario_path('basic.wad')
        game.set_doom_map('map01')
        game.set_screen_resolution(ScreenResolution.RES_160X120)
        game.set_screen_format(ScreenFormat.GRAY8)
        game.set_render_hud(False)
        game.set_render_crosshair(False)
        game.set_render_weapon(True)
        game.set_render_decals(False)
        game.set_render_particles(False)
        game.add_available_button(Button.MOVE_LEFT)
        game.add_available_button(Button.MOVE_RIGHT)
        game.add_available_button(Button.ATTACK)
        game.add_available_game_variable(GameVariable.AMMO2)
        game.add_available_game_variable(GameVariable.POSITION_X)
        game.add_available_game_variable(GameVariable.POSITION_Y)
        game.set_episode_timeout(300)
        game.set_episode_start_time(10)
        game.set_window_visible(False)
        game.set_sound_enabled(False)
        game.set_living_reward(-1)
        game.set_mode(Mode.PLAYER)
        game.init()
        self.actions = np.identity(a_size, dtype=bool).tolist()
        # end Doom setup
        
        self.env = game
        
    def train(self, rollout, sess, gamma, bootstrap_value):
        rollout = np.array(rollout)
        observations = rollout[:, 0]
        actions = rollout[:, 1]
        rewards = rollout[:, 2]
        next_observations = rollout[:, 3]
        values = rollout[:, 5]
        
        # Take the rewards and values from the rollout, and use them to
        # generate the advantage and discounted returns. 
        # The advantage function uses "Generalized Advantage Estimation"
        self.rewards_plus = np.asarray(rewards.tolist() + [bootstrap_value])
        discounted_rewards = discount(self.rewards_plus, gamma)[:-1]
        self.value_plus = np.asarray(values.tolist() + [bootstrap_value])
        advantages = rewards + gamma * self.value_plus[1:] - self.value_plus[:-1]
        advantages = discount(advantages, gamma)
        
        # Update the global network using gradients from loss
        # Generate network statistics to periodically save
        feed_dict = {
            self.local_ac.target_v: discounted_rewards,
            self.local_ac.inputs: np.vstack(observations),
            self.local_ac.actions: actions,
            self.local_ac.advantages: advantages,
            self.local_ac.state_in[0]: self.batch_rnn_state[0],
            self.local_ac.state_in[1]: self.batch_rnn_state[1]
        }
        v1, p1, e1, g_n, v_n, self.batch_rnn_state, _ = \
            sess.run([self.local_ac.value_loss, 
                      self.local_ac.policy_loss, 
                      self.local_ac.entropy, 
                      self.local_ac.grad_norms, 
                      self.local_ac.var_norms, 
                      self.local_ac.state_out, 
                      self.local_ac.apply_grads
                     ], feed_dict = feed_dict)
        rollout_length = len(rollout)
        
        return v1 / rollout_length, p1 / rollout_length, e1 / rollout_length, g_n, v_n
    
    def work(self, max_episode_length, gamma, sess, coord, saver):
        episode_count = sess.run(self.global_episodes)
        n_steps = 0
        print('Starting worker', str(self.number))
        with sess.as_default(), sess.graph.as_default():
            while not coord.should_stop():
                sess.run(self.update_local_ops)
                episode_buffer, episode_values, episode_frames = [], [], []
                episode_reward = 0
                episode_step_count = 0
                done = False
                
                self.env.new_episode()
                s = self.env.get_state().screen_buffer
                episode_frames.append(s)
                s = process_frame(s)
                rnn_state = self.local_ac.state_init
                self.batch_rnn_state = rnn_state
                while not self.env.is_episode_finished():
                    # Take an action using probabilities from policy network output
                    a_dist, v, rnn_state = sess.run([self.local_ac.policy, 
                                                     self.local_ac.value, 
                                                     self.local_ac.state_out], feed_dict={
                        self.local_ac.inputs: [s],
                        self.local_ac.state_in[0]: rnn_state[0],
                        self.local_ac.state_in[1]: rnn_state[1]
                    })
                    a = np.random.choice(a_dist[0], p=a_dist[0])
                    a = np.argmax(a_dist == a)
                    
                    reward = self.env.make_action(self.actions[a]) / 100.
                    done = self.env.is_episode_finished()
                    if done:
                        s1 = s
                    else:
                        s1 = self.env.get_state().screen_buffer
                        episode_frames.append(s1)
                        s1 = process_frame(s1)
                        
                    episode_buffer.append([s, a, reward, s1, done, v[0, 0]])
                    episode_values.append(v[0, 0])
                    
                    episode_reward += reward
                    s = s1
                    n_steps += 1
                    episode_step_count += 1
                    
                    # If the episode hasn't ended, but the experience buffer is full, then we
                    # make an update step using that experience rollout.
                    if len(episode_buffer) == 30 and not done and episode_step_count != max_episode_length - 1:
                        # Since we don't know what the true final return is,
                        # we "bootstrap" from our current value estimation.
                        v1 = sess.run(self.local_ac.value, feed_dict={
                            self.local_ac.inputs: [s],
                            self.local_ac.state_in[0]: rnn_state[0],
                            self.local_ac.state_in[1]: rnn_state[1]
                        })[0, 0]
                        v1, p1, e1, g_n, v_n = self.train(episode_buffer, sess, gamma, v1)
                        episode_buffer = []
                        sess.run(self.update_local_ops)
                    if done:
                        break
                        
                self.episode_rewards.append(episode_reward)
                self.episode_lengths.append(episode_step_count)
                self.episode_mean_values.append(np.mean(episode_values))
                
                # Update the network using the episode buffer at the end of the episode
                if len(episode_buffer) != 0:
                    v1, p1, e1, g_n, v_n = self.train(episode_buffer, sess, gamma, 0.)
                    
                # Periodically save episode gifs, model parameters, and summary statistics
                if episode_count % 5 == 0 and episode_count != 0:
                    if self.name == 'worker_0' and episode_count % 25 == 0:
                        time_per_step = 0.05
                        images = np.array(episode_frames)
                        make_gif(images, './frames/image{}.gif'.format(episode_count), 
                                 duration=len(images) * time_per_step, 
                                 true_image=True, salience=False)
                        
                if episode_count % 250 == 0 and self.name == 'worker_0':
                    saver.save(sess, '{}/model-{}'.format(self.model_path, episode_count))
                    #print('Model saved')
                    
                mean_reward = np.mean(self.episode_rewards[-5:])
                mean_length = np.mean(self.episode_lengths[-5:])
                mean_value = np.mean(self.episode_mean_values[-5:])
                summary = tf.Summary()
                summary.value.add(tag='Perf/Reward', simple_value=float(mean_reward))
                summary.value.add(tag='Perf/Length', simple_value=float(mean_length))
                summary.value.add(tag='Perf/Value', simple_value=float(mean_value))
                summary.value.add(tag='Loss/Value', simple_value=float(v1))
                summary.value.add(tag='Loss/Policy', simple_value=float(p1))
                summary.value.add(tag='Loss/Entropy', simple_value=float(e1))
                summary.value.add(tag='Loss/Grad Norm', simple_value=float(g_n))
                summary.value.add(tag='Loss/Var Norm', simple_value=float(v_n))
                self.summary_writer.add_summary(summary, episode_count)
                self.summary_writer.flush()
                
            if self.name == 'worker_0':
                sess.run(self.increment)
                
            episode_count += 1

In [37]:
# hyperparams:

max_episode_length = 300
gamma = 0.99  # discount rate for advantage estimation and reward discounting
s_size = 7056  # observations are greyscale frames, shape[84, 84, 1]
a_size = 3  # Agent can move Left, Right, or Fire
load_model = False
model_path = './model'

In [38]:
tf.reset_default_graph()

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
# Create a directory to save episode playback gifs
if not os.path.exists('./frames'):
    os.makedirs('./frames')
    
with tf.device('/cpu:0'):
    global_episodes = tf.Variable(0, dtype=tf.int32, name='global_episodes', trainable=False)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
    
    # Generate global network
    master_network = ACNetwork(s_size, a_size, 'global', None)
    
    # Set workers to number of available CPU threads
    n_workers = multiprocessing.cpu_count()
    workers = []
    
    # Create worker classes
    for i in range(n_workers):
        workers.append(Worker(DoomGame(), i, s_size, a_size, optimizer, model_path, global_episodes))
        
    saver = tf.train.Saver(max_to_keep=5)
    
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model:
        print('Loading model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
        
    # This is where the asynchronous magic happens.
    # Start the "work" process for each worker in a separate thread
    worker_threads = []
    for worker in workers:
        work = lambda: worker.work(max_episode_length, gamma, sess, coord, saver)
        t = threading.Thread(target=(work))
        t.start()
        sleep(0.5)
        worker_threads.append(t)
        
    coord.join(worker_threads)

Starting worker 0
Starting worker 1
Model saved
Starting worker 2
Starting worker 3
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Mode

Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Mode

Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Mode

Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Mode

Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Mode

...

Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Mode

Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Mode

Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Model saved
Mode

Exception in thread Thread-36:
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1322, in _do_call
    return fn(*args)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1307, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 1409, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Run call was cancelled

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-38-197483458015>", line 40, in <lambda>
    work = lambda: worker.work(max_episode_length, gamma, s

KeyboardInterrupt: 